# 10. 다양한 데이터 전처리 기법

**EDA를 통해 도출된 데이터 인사이트를 토대로, 효과적인 Feature Engineering을 위해 사용하는 Encoding, Scaling, Feature Selection 등의 전처리 기법을 실습해 본다.**

## 10-1. 들어가며

```bash
$ mkdir -p ~/aiffel/data_preprocess/
$ ln -s ~/data/ ~/aiffel/data_preprocess/
```

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

print("👽 Hello.")

In [ ]:
import os

csv_file_path = os.getenv('HOME')+'/aiffel/data_preprocess/data/trade.csv'
trade = pd.read_csv(csv_file_path) 
trade.head()

## 10-2. 결측치(Missing Data)

In [ ]:
print('전체 데이터 건수:', len(trade))

In [ ]:
print('컬럼별 결측치 개수')
len(trade) - trade.count()

In [ ]:
trade = trade.drop('기타사항', axis=1)
trade.head()

In [ ]:
trade.isnull()

In [ ]:
trade.isnull().any(axis=1)

In [ ]:
trade[trade.isnull().any(axis=1)]

In [ ]:
trade.dropna(how='all', subset=['수출건수', '수출금액', '수입건수', '수입금액', '무역수지'], inplace=True)
print("👽 It's okay, no biggie.")

In [ ]:
trade[trade.isnull().any(axis=1)]

In [ ]:
trade.loc[[188, 191, 194]]

In [ ]:
trade.loc[191, '수출금액'] = (trade.loc[188, '수출금액'] + trade.loc[194, '수출금액'] )/2
trade.loc[[191]]

In [ ]:
trade.loc[191, '무역수지'] = trade.loc[191, '수출금액'] - trade.loc[191, '수입금액'] 
trade.loc[[191]]

In [ ]:
#-- 아래에 코드를 작성해 주세요. --#
trade.loc[191, '수출금액'] = (np.median(trade.loc[188, '수출금액']) + np.median(trade.loc[194, '수출금액']))/2

trade.loc[191, '무역수지'] = (trade.loc[191, '수출금액'] - trade.loc[191, '수입금액'])/2
trade.loc[[191]]

## 10-3. 중복된 데이터

In [ ]:
trade.duplicated()

In [ ]:
trade[trade.duplicated()]

In [ ]:
trade[(trade['기간']=='2020년 03월')&(trade['국가명']=='중국')]

In [ ]:
trade.drop_duplicates(inplace=True)
print("👽 It's okay, no biggie.")

In [ ]:
df = pd.DataFrame({'id':['001', '002', '003', '004', '002'], 
                   'name':['Park Yun', 'Kim Sung', 'Park Jin', 'Lee Han', 'Kim Min']})
df

In [ ]:
df.drop_duplicates(subset=['id'], keep='last')

## 10-4. 이상치(Outlier)

In [ ]:
def outlier(df, col, z):
    return df[abs(df[col] - np.mean(df[col]))/np.std(df[col])>z].index
print("👽 It's okay, no biggie.")

In [ ]:
trade.loc[outlier(trade, '무역수지', 1.5)]

In [ ]:
trade.loc[outlier(trade, '무역수지', 2)]

In [ ]:
trade.loc[outlier(trade, '무역수지', 3)]

In [ ]:
def not_outlier(df, col, z):
    return df[abs(df[col] - np.mean(df[col]))/np.std(df[col]) <= z].index
print("👽 It's okay, no biggie.")

In [ ]:
trade.loc[not_outlier(trade, '무역수지', 1.5)]

In [ ]:
np.random.seed(2020)
data = np.random.randn(100)  # 평균 0, 표준편차 1의 분포에서 100개의 숫자를 샘플링한 데이터 생성
data = np.concatenate((data, np.array([8, 10, -3, -5])))      # [8, 10, -3, -5])를 데이터 뒤에 추가함
data

In [ ]:
fig, ax = plt.subplots()
ax.boxplot(data)
plt.show()

In [ ]:
Q3, Q1 = np.percentile(data, [75 ,25])
IQR = Q3 - Q1
IQR

In [ ]:
data[(Q1-1.5*IQR > data)|(Q3+1.5*IQR < data)]

In [ ]:
def outlier2(df, col):
    # [[YOUR CODE]]
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1
    return df[(df[col] < q1-1.5*iqr)|(df[col] > q3+1.5*iqr)]

outlier2(trade, '무역수지')

## 10-5. 정규화(Normalization)

In [ ]:
# 정규분포를 따라 랜덤하게 데이터 x를 생성합니다. 
x = pd.DataFrame({'A': np.random.randn(100)*4+4,
                 'B': np.random.randn(100)-1})
x

In [ ]:
# 데이터 x를 Standardization 기법으로 정규화합니다. 
x_standardization = (x - x.mean())/x.std()
x_standardization

In [ ]:
# 데이터 x를 min-max scaling 기법으로 정규화합니다. 
x_min_max = (x-x.min())/(x.max()-x.min())
x_min_max

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(12, 4),
                        gridspec_kw={'width_ratios': [2, 1]})

axs[0].scatter(x['A'], x['B'])
axs[0].set_xlim(-5, 15)
axs[0].set_ylim(-5, 5)
axs[0].axvline(c='grey', lw=1)
axs[0].axhline(c='grey', lw=1)
axs[0].set_title('Original Data')

axs[1].scatter(x_standardization['A'], x_standardization['B'])
axs[1].set_xlim(-5, 5)
axs[1].set_ylim(-5, 5)
axs[1].axvline(c='grey', lw=1)
axs[1].axhline(c='grey', lw=1)
axs[1].set_title('Data after standardization')

plt.show()

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(12, 4),
                        gridspec_kw={'width_ratios': [2, 1]})

axs[0].scatter(x['A'], x['B'])
axs[0].set_xlim(-5, 15)
axs[0].set_ylim(-5, 5)
axs[0].axvline(c='grey', lw=1)
axs[0].axhline(c='grey', lw=1)
axs[0].set_title('Original Data')

axs[1].scatter(x_min_max['A'], x_min_max['B'])
axs[1].set_xlim(-5, 5)
axs[1].set_ylim(-5, 5)
axs[1].axvline(c='grey', lw=1)
axs[1].axhline(c='grey', lw=1)
axs[1].set_title('Data after min-max scaling')

plt.show()

In [ ]:
# trade 데이터를 Standardization 기법으로 정규화합니다. 
cols = ['수출건수', '수출금액', '수입건수', '수입금액', '무역수지']
trade_Standardization= (trade[cols]-trade[cols].mean())/trade[cols].std()
trade_Standardization.head()

In [ ]:
trade_Standardization.describe()

In [ ]:
# trade 데이터를 min-max scaling 기법으로 정규화합니다. 
trade[cols] = (trade[cols]-trade[cols].min())/(trade[cols].max()-trade[cols].min())
trade.head()

In [ ]:
trade.describe()

In [ ]:
train = pd.DataFrame([[10, -10], [30, 10], [50, 0]])
test = pd.DataFrame([[0, 1], [10, 10]])
print("👽 It's okay, no biggie.")

In [ ]:
train_min = train.min()
train_max = train.max()

train_min_max = (train - train_min)/(train_max - train_min)
test_min_max =  (test - train_min)/(train_max - train_min)    # test를 min-max scaling할 때도 train 정규화 기준으로 수행
print("💫 It's okay, no biggie...")

In [ ]:
train_min_max

In [ ]:
test_min_max

In [ ]:
from sklearn.preprocessing import MinMaxScaler
train = [[10, -10], [30, 10], [50, 0]]
test = [[0, 1]]
scaler = MinMaxScaler()
print("👽 It's okay, no biggie.")

In [ ]:
scaler.fit_transform(train)

In [ ]:
scaler.transform(test)

## 10-6. 원-핫 인코딩(One-Hot Encoding)

In [ ]:
#trade 데이터의 국가명 컬럼 원본
print(trade['국가명'].head())  

# get_dummies를 통해 국가명 원-핫 인코딩
country = pd.get_dummies(trade['국가명'])
country.head()

In [ ]:
trade = pd.concat([trade, country], axis=1)
trade.head()

In [ ]:
trade.drop(['국가명'], axis=1, inplace=True)
trade.head()

## 10-7. 구간화(Binning)

In [ ]:
salary = pd.Series([4300, 8370, 1750, 3830, 1840, 4220, 3020, 2290, 4740, 4600, 
                    2860, 3400, 4800, 4470, 2440, 4530, 4850, 4850, 4760, 4500, 
                    4640, 3000, 1880, 4880, 2240, 4750, 2750, 2810, 3100, 4290, 
                    1540, 2870, 1780, 4670, 4150, 2010, 3580, 1610, 2930, 4300, 
                    2740, 1680, 3490, 4350, 1680, 6420, 8740, 8980, 9080, 3990, 
                    4960, 3700, 9600, 9330, 5600, 4100, 1770, 8280, 3120, 1950, 
                    4210, 2020, 3820, 3170, 6330, 2570, 6940, 8610, 5060, 6370,
                    9080, 3760, 8060, 2500, 4660, 1770, 9220, 3380, 2490, 3450, 
                    1960, 7210, 5810, 9450, 8910, 3470, 7350, 8410, 7520, 9610, 
                    5150, 2630, 5610, 2750, 7050, 3350, 9450, 7140, 4170, 3090])
print("👽 Almost there..")

In [ ]:
salary.hist()

In [ ]:
bins = [0, 2000, 4000, 6000, 8000, 10000]
print("👽 Almost there..")

In [ ]:
ctg = pd.cut(salary, bins=bins)
ctg

In [ ]:
print('salary[0]:', salary[0])
print('salary[0]가 속한 카테고리:', ctg[0])

In [ ]:
ctg.value_counts().sort_index()

In [ ]:
ctg = pd.cut(salary, bins=6)
ctg

In [ ]:
ctg.value_counts().sort_index()

In [ ]:
ctg = pd.qcut(salary, q=5)
ctg

In [ ]:
print(ctg.value_counts().sort_index())
print(".\n.\n🛸 Well done!")